# Air Brakes Example


In [ ]:
from rocketpy import Environment, SolidMotor, Rocket, Flight, Function

In [ ]:
%matplotlib widget

In [ ]:
env = Environment(latitude=32.990254, longitude=-106.974998, elevation=1400)

In [ ]:
Pro75M1670 = SolidMotor(
    thrust_source="../../data/motors/Cesaroni_M1670.eng",
    dry_mass=1.815,
    dry_inertia=(0.125, 0.125, 0.002),
    nozzle_radius=33 / 1000,
    grain_number=5,
    grain_density=1815,
    grain_outer_radius=33 / 1000,
    grain_initial_inner_radius=15 / 1000,
    grain_initial_height=120 / 1000,
    grain_separation=5 / 1000,
    grains_center_of_mass_position=0.397,
    center_of_dry_mass_position=0.317,
    nozzle_position=0,
    burn_time=3.9,
    throat_radius=11 / 1000,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)

In [ ]:
calisto = Rocket(
    radius=127 / 2000,
    mass=14.426,
    inertia=(6.321, 6.321, 0.034),
    power_off_drag="../../data/calisto/powerOffDragCurve.csv",
    power_on_drag="../../data/calisto/powerOnDragCurve.csv",
    center_of_mass_without_motor=0,
    coordinate_system_orientation="tail_to_nose",
)

rail_buttons = calisto.set_rail_buttons(
    upper_button_position=0.0818,
    lower_button_position=-0.618,
    angular_position=45,
)

calisto.add_motor(Pro75M1670, position=-1.255)

nose_cone = calisto.add_nose(length=0.55829, kind="vonKarman", position=1.278)

fin_set = calisto.add_trapezoidal_fins(
    n=4,
    root_chord=0.120,
    tip_chord=0.060,
    span=0.110,
    position=-1.04956,
    cant_angle=0.5,
    airfoil=("../../data/calisto/NACA0012-radians.csv", "radians"),
)

tail = calisto.add_tail(
    top_radius=0.0635, bottom_radius=0.0435, length=0.060, position=-1.194656
)

In [ ]:
def controller_function(time, sampling_rate, state, state_history, air_brakes):
    # state = [x, y, z, v_x, v_y, v_z, e0, e1, e2, e3, w_x, w_y, w_z]
    z = state[2]
    vz = state[5]

    # Get previous state from state_history
    previous_state = state_history[-1]
    previous_vz = previous_state[5]
    
    # Check if the rocket has reached burnout
    if time > Pro75M1670.burn_out_time:
        # If below 1500 meters, air_brakes are not deployed
        if z < 1500 + env.elevation:
            air_brakes.set_deployed_level(0)

        # Else calculate the deployed level
        else:
            new_deployed_level = (
                air_brakes.deployed_level + 0.1 * vz + 0.01 * previous_vz**2
            )

            # Limiting the speed of the air_brakes to 0.1 per second
            # Since this function is called every 1/sampling_rate seconds
            # the max change in deployed level per call is 0.1/sampling_rate
            if new_deployed_level > air_brakes.deployed_level + 0.2 / sampling_rate:
                new_deployed_level = air_brakes.deployed_level + 0.2 / sampling_rate
            elif new_deployed_level < air_brakes.deployed_level - 0.2 / sampling_rate:
                new_deployed_level = air_brakes.deployed_level - 0.2 / sampling_rate
            else:
                new_deployed_level = air_brakes.deployed_level

            air_brakes.set_deployed_level(new_deployed_level)

In [ ]:
air_brakes, controller = calisto.add_air_brakes(
    drag_coefficient_curve="../../data/calisto/air_brakes_cd.csv",
    controller_function=controller_function,
    sampling_rate=10,
    reference_area=None,
    clamp=True,
    name="AirBrakes",
    controller_name="AirBrakes Controller",
)

In [ ]:
air_brakes.all_info()

In [ ]:
air_brakes.drag_coefficient == 'Function from R2 to R1 : (Deployed Level, Mach) → (Drag Coefficient)'

In [ ]:
test_flight = Flight(
    rocket=calisto,
    environment=env,
    rail_length=5.2,
    inclination=85,
    heading=0,
    time_overshoot=False,
    terminate_on_apogee=True,
)

## Analyzing the Results

Now we can see some plots from our air brakes:

In [ ]:
air_brakes.deployed_level_by_time.plot(force_data=True)
air_brakes.drag_coefficient_by_time.plot(force_data=True)

And of course, the simulation results:

In [ ]:
test_flight.altitude()
test_flight.vz()

In [ ]:
test_flight.all_info()